In [17]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("Dataset .csv")
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [3]:
# Create a map centered around the median latitude and longitude of the dataset
center_lat = df['Latitude'].median()
center_long = df['Longitude'].median()

restaurant_map = folium.Map(location=[center_lat, center_long], zoom_start=12)

In [4]:
# Add marker cluster to the map
marker_cluster = MarkerCluster().add_to(restaurant_map)

In [5]:
# Add points to the marker cluster
for idx, row in df.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']],
                  popup=f"{row['Restaurant Name']} ({row['Cuisines']})\nRating: {row['Aggregate rating']}").add_to(marker_cluster)


In [6]:
# Save the map to an HTML file
map_path = 'restaurants_map.html'
restaurant_map.save(map_path)

map_path

'restaurants_map.html'

In [7]:
# Group by city and count the number of restaurants in each city
city_group = df.groupby('City').size().reset_index(name='Number of Restaurants')


In [8]:
# Display the top cities with the highest concentration of restaurants
top_cities = city_group.sort_values(by='Number of Restaurants', ascending=False).head(10)
top_cities

,City,Number of Restaurants
88,New Delhi,5473
50,Gurgaon,1118
89,Noida,1080
43,Faridabad,251
48,Ghaziabad,25
17,Bhubaneshwar,21
69,Lucknow,21
5,Amritsar,21
2,Ahmedabad,21
51,Guwahati,21


In [9]:
# Function to safely get the most common cuisine
def most_common_cuisine(cuisines):
    if cuisines.dropna().empty:
        return 'Unknown'
    return cuisines.value_counts().idxmax()

In [10]:
# Calculate average ratings, most common cuisines, and average price range for each city
average_ratings = df.groupby('City')['Aggregate rating'].mean().reset_index(name='Average Rating')

In [11]:
most_common_cuisines = df.groupby('City')['Cuisines'].apply(lambda x: most_common_cuisine(x)).reset_index(name='Most Common Cuisine')

In [12]:
average_price_range = df.groupby('City')['Price range'].mean().reset_index(name='Average Price Range')


In [13]:
# Merge the statistics into a single dataframe
city_stats = average_ratings.merge(most_common_cuisines, on='City').merge(average_price_range, on='City')


In [14]:
# Merge with the top cities to get the desired statistics
top_city_stats = top_cities.merge(city_stats, on='City')

In [15]:
# Display the results
print(top_city_stats)

           City  Number of Restaurants  Average Rating  \
0     New Delhi                   5473        2.438845   
1       Gurgaon                   1118        2.651431   
2         Noida                   1080        2.036204   
3     Faridabad                    251        1.866932   
4     Ghaziabad                     25        2.852000   
5  Bhubaneshwar                     21        3.980952   
6       Lucknow                     21        4.195238   
7      Amritsar                     21        3.685714   
8     Ahmedabad                     21        4.161905   
9      Guwahati                     21        4.190476   

                                 Most Common Cuisine  Average Price Range  
0                                       North Indian             1.621597  
1                                       North Indian             1.855993  
2                                       North Indian             1.601852  
3                                       North Indian     

In [16]:
# Output the path to the saved map
print(f"Map saved to: {map_path}")

Map saved to: restaurants_map.html
